In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass() # 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

In [2]:
os.environ["LANGSMITH_API_KEY"]

'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'

## Download Ollama
https://ollama.com/library/mxbai-embed-large

In [3]:
# %pip install -qU langchain-ollama
# %pip install -U ollama

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
    # other params...
)

In [6]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

ResponseError: model "llama3.2" not found, try pulling it first (status code: 404)